# AWS CloudTrail
- AWS CloudTrail은 AWS 계정의 커버넌스, 규정 준수, 운영 감사, 위험 감사를 지원하는 서비스입니다. CloudTrail을 사용하면 AWS 인프라에서 게정 활동과 관련된 작업을 기록하고 지속적으로 모니터링하며 보관할 수 있습니다. CloudTrail은 AWS Management Console, AWS SDK, 명령줄 도구 및 기타 AWS 서비스를 통해 수행된 작업을 비롯하여 AWS 계정 활동의 이벤트 기록을 제공합니다.
- AWS의 보안 및 감사를 위한 서비스 : 감시 (CCTV)
- AWS의 여러 서비스에 대한 활동 이벤트 로그 등을 제공
    - 이벤트 예시 : EC2 종료, S3 버킷 생성, IAM 사용자 삭제 등
    - 이벤트의 시간 및 결과, 에러, 사용 인증 정보 등을 기록
    - AWS CLI, 콘솔 이용, API 호출 등 모든 이벤트가 대상
- 기본적으로 90일의 이벤트 로그를 무료로 저장
    - 이 이상의 기록은 트레일 생성 필요
- 실시간 기록이 아니며 지연 시간 발생 (약 5분)

## AWS CloudTrail Trail
- S3 기반의 이벤트 로그 수집 단위
    - S3에 저장되기 때문에 S3 버킷을 생성하고 로그 저장 대상으로 지정할 필요가 있다.
- 리전 기반, 단 다중 리전 혹은 단일 리전 선택 가능
    - 다중 리전일 경우 모든 리전의 로그를 한 곳으로 모아 관리 가능
        - 신규 리전이 추가될 경우 자동으로 로깅 활성화
    - 글로벌 서비스의 경우 us-east-1 리전에서 로깅(IAM, STS, CloudFront)
        - 즉 다중 리전 로깅 혹은 us-east-1에 Trail을 생성해야 한다
- 별도로 AWS의 다양한 서비스(EventBridge, CloudWatch)로 이벤트 전달 설정 가능
    - Metric Filter / 이벤트 트리거 가능
    - 5분 이상 지연시간이 생길 가능성을 염두에 두어야 한다

## AWS CloudTrail Event
- AWS 계정의 활동 내용
    - 몇 Data API의 경우 수동으로 활성화 해줘야 로그가 남는다 (S3, Lambda, DynamoDB)
    - JSON 형식으로 데이터 저장
        - Athena 등으로 분석 가능
- 총 세가지 종류
    - Management Event : AWS 계정의 관리를 위한 이벤트
        - 예 : IAM 역할 부여, VPC 생성/삭제, 서브넷 생성/삭제, Trail 생성, 로그인 이벤트 등
        - 기본 로깅 대상
    - Data Event : 리소스의 동작과 관련된 데이터
        - 예 : S3의 Object 동작(Get, Delete, Put), Lambda 함수 호출, SNS 호출 등
        - 별도로 로깅 활성화 필요 + 추가 비용 발생
    - Insight Event : CloudTrail에서 별도로 생성하는 정상적이지 않은 상황에 대한 감지 이벤트
        - 예 : 평소와 다른 S3버킷 100개 삭제, Deny된 인스턴스 프로비전 요청이 분당 수십 개 발생
        - 별도로 로깅 활성화 필요 + 추가 비용 발생

### 데모
- 클라우드 트레일 트레일 생성 및 모니터링 확인
    - 트레일 생성, 데이터 이벤트 수집 활성화
    - 클라우드셸에서 EC2 정보 조회(Describe-Instance) API 호출 및 로깅 확인
    - S3 오브젝트 생성/요청 이벤트 확인

### 데모 순서
1. CloudTrail로 이동
    - 이벤트 기록 조회해보자
    - 추적(트레일) 생성
        - Demo-My-Trail
        - 새 S3 버킷 생성
        - KMS 별칭 입력 : Demo-CloudTrail-KMS
        - CloudWatch Logs 활성화
            - 역할 이름 : CloudTrailRoleForCloudWatch-My-Demo-Trail
        - 이벤트 유형 : 데이터 이벤트 추가
        - 데이터 이벤트 : S3
2. 인스턴스 프로비전, 트레일에서 확인해보자
    - 클라우드 트레일 : 이벤트 기록으로 이동해보면 기록이 보인다
    - 인스턴스 삭제 : 종료 내역도 확인 가능
3. S3 이벤트 로깅 확인해보기
    - 아무 버킷에 파일 업로드 : 이벤트 기록으로 이동, 소스를 S3로 필터링
    - 오브젝트와 관련된 내용은 보이지 않는다. : S3 로그 버킷의 Trail Json 파일에서 확인할 수 있음 or Athena 테이블로 확인 가능
    - 클라우드트레일 Lake도 활용 가능
4. S3 버킷, 트레일 삭제, 버킷 내 로그도 삭제
        

## CloudTrail VS CloudWatch
- CloudTrail : AWS를 감사(Audit)하기 위한 서비스 : 감시(CCTV)
    - AWS의 모든 서비스가 사용될 때마다 사용 로그를 저장
    - AWS가 언제 어디서 누구에 의해 사용되었는가?
    - 단순하게 AWS 사용 로그만 저장
- CloudWatch : AWS를 모니터링하기 위한 서비스 : 퍼포먼스 체크
    - AWS의 서비스 뿐 아니라 어플리케이션의 로그 및 동작 로그 취합
    - 어플리케이션이 어떻게 동작하였는가? 무슨 버그가 있었는가? 메모리는 얼마나 소모되었는가?
    - 대시보드 및 알람 등 모니터링을 위한 서비스 제공